# Hate Speech Project
## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#h">Hypothesis Generation</a></li>
<li><a href="#i">Import Packages and Loading Data</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#u">Univariate Exploration</a></li>
<li><a href="#b">Bivariate Exploration</a></li>
<li><a href="#m">Missing Value Treatment</a></li>
<li><a href="#o">Outliers Removal</a></li>
<li><a href="#f">Feature Engineering</a></li>
<li><a href="#e">Encoding</a></li>
<li><a href="#s">Scaling</a></li>
<li><a href="#sp">Splitting</a></li>
<li><a href="#mo">Model</a></li>

</ul>

<a id='intro'></a>
## Introduction

### Project Description 
The term hate speech is understood as any type of verbal, written or behavioural communication that attacks or uses derogatory or discriminatory language against a person or group based on what they are, in other words, based on their religion, ethnicity, nationality, race, colour, ancestry, sex or another identity factor. In this problem, We will take you through a hate speech detection model with Machine Learning and Python.

Hate Speech Detection is generally a task of sentiment classification. So for training, a model that can classify hate speech from a certain piece of text can be achieved by training it on a data that is generally used to classify sentiments. So for the task of hate speech detection model, We will use the Twitter tweets to identify tweets containing  Hate speech.

In [1]:
import pandas as pd


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print("Train shape:",train.shape)
print("Test shape:",test.shape)

Train shape: (31962, 3)
Test shape: (17197, 2)


In [6]:
31962+17197


49159

In [4]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
